In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName("ETL")\
        .config('spark.jars.packages', 'org.postgresql:postgresql:42.6.2') \
        .getOrCreate()
sqlContext = SparkSession(spark)
#Dont Show warning only error
spark.sparkContext.setLogLevel("ERROR")

In [3]:
user_df = spark.read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://103.245.236.247:5432/e-commerce") \
    .option("driver", "org.postgresql.Driver")\
    .option("user", "traianthai") \
    .option("password", "123123") \
    .option("dbtable", "public.\"user\"") \
    .load()


In [4]:
user_df.show()

+---+--------------------+--------------------+---------+--------------------+----------+------------+--------+------+-------------------+-------------------+-----+------+--------------------+-----------------+---------+-------------+
| id|           createdAt|           updatedAt|deletedAt|               email|  password|   firstName|lastName|gender|        dateOfBirth|              phone|state|roleId|              avatar|         nickName|addressId|    numberCMT|
+---+--------------------+--------------------+---------+--------------------+----------+------------+--------+------+-------------------+-------------------+-----+------+--------------------+-----------------+---------+-------------+
|200|2024-05-14 20:29:...|2024-05-14 20:29:...|     NULL|anh111.cty@gmail.com|    123123|Trần Văn Đức|     Anh|   Nam|1965-09-01 08:00:00|           18375156|    1|     1|https://cdn.thoit...|        thao112CT|     NULL|2312137047291|
|150|2024-05-14 20:29:...|2024-05-14 20:29:...|     NULL|man

In [6]:
from pyspark.sql.functions import col

# Tạo DataFrame mới chỉ chứa cột "id" và "name"
user_df = user_df.select(col("id"), col("firstName"), col("lastName"), col("email"), col("phone"))

# Hiển thị DataFrame mới
user_df.show()


+---+------------+--------+--------------------+-------------------+
| id|   firstName|lastName|               email|              phone|
+---+------------+--------+--------------------+-------------------+
|200|Trần Văn Đức|     Anh|anh111.cty@gmail.com|           18375156|
|150|  Nguyễn Văn|    Mạnh|manh61.cty@gmail.com|           78778639|
|  3|       admin|   admin|  admin@ecommerce.io|               NULL|
|  2|        shop|    test|traianthai@gmail.com|               NULL|
|  4|       Maria|   Smith|marisacampos@hotm...| (355)090-3692x8791|
|  5|     Charles|   Smith|michelleobrien@ya...|(088)622-6104x48167|
|  6|       Cathy|   Boyer|veronica60@vincen...|        03093376553|
|  7|   Nathaniel|Mcintyre|ramirezgary@yahoo...|1-313-282-1839x1537|
|  8|     Abigail|  Howell|adamsjonathan@agu...|(077)519-4387x33017|
|  9|       Cindy| Sampson|    ogreen@yahoo.com|  328.504.2836x6889|
| 10|        Ruth|    Gray|  ashley48@hill.info|      (994)140-1207|
| 11|    Kathleen| Johnson|alyssac

In [8]:
# Ghi DataFrame dim_time vào bảng "dim_time" trong schema "demo"
user_df.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://103.245.236.247:5432/data_warehouse") \
    .option("dbtable", "public.\"Dim_User\"") \
    .option("user", "traianthai") \
    .option("password", "123123") \
    .mode("overwrite") \
    .save()


In [9]:
# Đóng SparkSession
spark.stop()